# Quick introduction to neural networks as functions approximators

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
import torch.nn as nn
import torch

## Define a function that we want to approximate

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f_x = lambda x: np.sin(x) - (x**2 / 40)

In [ ]:
X = np.arange(-5, 5, .1)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(f_x(X))
plt.tight_layout()
plt.show()

## Use a NN to learn the function

In [ ]:
class SigmoidNeuralNet(nn.Module):
    """Double-layer neural network with sigmoid activation."""
    
    def __init__(self, num_features, nodes=4):
        super().__init__()
        self.layer1  = nn.Linear(in_features=num_features, out_features=nodes)
        self.layer2  = nn.Linear(in_features=nodes, out_features=num_features)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inputs):
        logits1 = self.layer1(inputs)
        activations = self.sigmoid(logits1)
        output = self.layer2(activations)
        return output

In [ ]:
class ReluNeuralNet(nn.Module):
    """Double-layer neural network with Relu activation."""
    
    def __init__(self, num_features, nodes=4):
        super().__init__()
        self.layer1  = nn.Linear(in_features=num_features, out_features=nodes)
        self.layer2  = nn.Linear(in_features=nodes, out_features=num_features)
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        logits1 = self.layer1(inputs)
        activations = self.relu(logits1)
        output = self.layer2(activations)
        return output

In [ ]:
inputs = torch.randint(-10, 10, (2000, 1)).float()
num_features = inputs.size(-1)
model = SigmoidNeuralNet(num_features, nodes=4)
#model = ReluNeuralNet(num_features)

In [ ]:
inputs

In [ ]:
prediction = model(inputs)

In [ ]:
prediction

### Training

In [ ]:
targets = f_x(inputs)

In [ ]:
targets

#### Loss function

In [ ]:
from torch.nn import MSELoss

In [ ]:
loss = MSELoss()
error = loss(prediction, targets)
error

#### Optimizer for gradient descent

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

#### Data management

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        self.x = inputs
        self.y = targets
        
    def __len__(self):
        return self.x.size(0)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [ ]:
dataset = CustomDataset(inputs, targets)
dataset[1]

**Dataloader**: Using a dataloader we can define how the network will read the data and how often parameters will be updated (batch approach)

In [ ]:
dataloader = DataLoader(dataset, batch_size=1, drop_last=False, shuffle=False)

In [ ]:
for i, batch in enumerate(dataloader):
    print(f"batch {i}: x: {batch[0]}, y: {batch[1]}")
    if i > 5:
        break

In [ ]:
dataloader = DataLoader(dataset, batch_size=10, drop_last=False, shuffle=False)

### Training routine

In [ ]:
epochs = 500

mean_error = []
predictions = []

run = list(range(epochs))

for epoch in tqdm(run):
    errors = []
    for batch in dataloader:
        # step 0: get the data and the correct target
        features, target = batch
        # step 1: use the model for prediction (forward pass)
        y_pred = model(features)
        # step 2: measure the error by comparing the predictions to the expected outputs
        error = loss(y_pred, target)
        # step 3: backprop the loss signal through the graph, notifying each parameter of its gradient
        error.backward()
        # step 4: update model parameters using the gradient
        optimizer.step() 
        # step 5: clear the previously computed gradients stored inside the model
        model.zero_grad()
        # step 6: reporting
        errors.append(error.item())
    mean_error.append(np.array(errors).mean())
    prediction_on_target = model(torch.tensor(X.reshape(-1, 1)).float()).detach().numpy().ravel()
    predictions.append(prediction_on_target)

### Learning process

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(mean_error)
plt.tight_layout()
plt.show()

### Fit procedure

In [ ]:
from IPython.display import clear_output

In [ ]:
for i, y_pred in enumerate(predictions[:300]):
    clear_output(wait=True)
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(f_x(X))
    ax.plot(y_pred)
    ax.set_xlabel('Epoch {}'.format(i))
    plt.tight_layout()
    plt.show()